## One round federated learning simulation

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


### Config variables for models and training


In [3]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
model_path = 'client/models/'
train_flag = True


### Load data


In [4]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


Files already downloaded and verified
Files already downloaded and verified


### Create two local models


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)



MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


### Train first local model


In [11]:
model1_name = 'fl_model1'

# Training
model1.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model1.parameters(), lr=learning_rate, weight_decay=reg)

train(model1, model_path, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3012
Epoch [1/1], Step [3/245], Loss: 2.2948
Epoch [1/1], Step [4/245], Loss: 2.2914
Epoch [1/1], Step [5/245], Loss: 2.2693
Epoch [1/1], Step [6/245], Loss: 2.2452
Epoch [1/1], Step [7/245], Loss: 2.2338
Epoch [1/1], Step [8/245], Loss: 2.2168
Epoch [1/1], Step [9/245], Loss: 2.1777
Epoch [1/1], Step [10/245], Loss: 2.1701
Epoch [1/1], Step [11/245], Loss: 2.1618
Epoch [1/1], Step [12/245], Loss: 2.1476
Epoch [1/1], Step [13/245], Loss: 2.0951
Epoch [1/1], Step [14/245], Loss: 2.1586
Epoch [1/1], Step [15/245], Loss: 2.0220
Epoch [1/1], Step [16/245], Loss: 2.0514
Epoch [1/1], Step [17/245], Loss: 2.1096
Epoch [1/1], Step [18/245], Loss: 2.0108
Epoch [1/1], Step [19/245], Loss: 2.0342
Epoch [1/1], Step [20/245], Loss: 2.0621
Epoch [1/1], Step [21/245], Loss: 2.0065
Epoch [1/1], Step [22/245], Loss: 2.0699
Epoch [1/1], Step [23/245], Loss: 2.0451
Epoch [1/1], Step [24/245], Loss: 1.9981
Epoch [1/1], Step [25/245

### Train second local model

In [12]:
model2_name = 'fl_model2'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, model_path, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3014
Epoch [1/1], Step [3/245], Loss: 2.2973
Epoch [1/1], Step [4/245], Loss: 2.2880
Epoch [1/1], Step [5/245], Loss: 2.2750
Epoch [1/1], Step [6/245], Loss: 2.2597
Epoch [1/1], Step [7/245], Loss: 2.2329
Epoch [1/1], Step [8/245], Loss: 2.2412
Epoch [1/1], Step [9/245], Loss: 2.1947
Epoch [1/1], Step [10/245], Loss: 2.2021
Epoch [1/1], Step [11/245], Loss: 2.1993
Epoch [1/1], Step [12/245], Loss: 2.1529
Epoch [1/1], Step [13/245], Loss: 2.1605
Epoch [1/1], Step [14/245], Loss: 2.0533
Epoch [1/1], Step [15/245], Loss: 2.0696
Epoch [1/1], Step [16/245], Loss: 2.1049
Epoch [1/1], Step [17/245], Loss: 2.0760
Epoch [1/1], Step [18/245], Loss: 2.0217
Epoch [1/1], Step [19/245], Loss: 2.0295
Epoch [1/1], Step [20/245], Loss: 2.0319
Epoch [1/1], Step [21/245], Loss: 2.0052
Epoch [1/1], Step [22/245], Loss: 2.0004
Epoch [1/1], Step [23/245], Loss: 1.9725
Epoch [1/1], Step [24/245], Loss: 2.0768
Epoch [1/1], Step [25/245

### Test the trained models

In [14]:
from os.path import join as pjoin
model_path = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(model_path, model1_name + '.ckpt'))
model1.load_state_dict(model_ckpt)
model = model1
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

model = model2
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 43.7 %
Accuracy of the network on the 1000 test images: 42.3 %



### Aggregate the local models


In [17]:
from client.utils import weights_zero_init
from client.aggregators import federated_aggregate
from os.path import join as pjoin


model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
# print(model3)

model_avg = federated_aggregate([model1, model3])

# model_avg = federated_aggregate([model1, model2])


# for parameter in model1.parameters():
#     print(parameter)
    
print(model1.state_dict()['layers.0.bias'])
print(model_avg.state_dict()['layers.0.bias'])
# the averaged model should be half the model1

    

tensor([ 8.1760e-02,  4.2900e-02,  5.2851e-02,  5.0147e-02,  5.7749e-02,
         3.8284e-02,  6.9570e-02,  3.4316e-02,  1.0920e-02,  5.5295e-02,
         4.3783e-02,  1.0032e-01,  6.2822e-02,  5.3711e-03,  8.9889e-02,
        -7.9435e-05,  8.7550e-02,  5.3161e-02, -4.7431e-03, -2.6418e-02,
         4.5759e-02,  1.8515e-02,  1.9627e-03,  5.3370e-02, -1.1729e-03,
         2.1432e-02,  7.6793e-02,  6.1534e-02,  1.6080e-02,  4.0207e-02,
         4.6392e-02, -1.2634e-02, -1.0254e-02,  3.3230e-02,  6.8993e-02,
         3.1953e-03, -4.5845e-02,  7.1283e-02,  9.0138e-02,  2.0860e-02,
        -9.9138e-03,  1.2532e-02,  6.2080e-02,  2.9840e-02,  7.1691e-02,
         1.7509e-02,  6.7909e-02,  4.6026e-02,  4.5743e-02,  5.9319e-02],
       device='cuda:0')
tensor([ 4.0880e-02,  2.1450e-02,  2.6425e-02,  2.5073e-02,  2.8874e-02,
         1.9142e-02,  3.4785e-02,  1.7158e-02,  5.4600e-03,  2.7648e-02,
         2.1891e-02,  5.0162e-02,  3.1411e-02,  2.6855e-03,  4.4945e-02,
        -3.9718e-05,  4.37


### Print the new global model


In [ ]:
for parameter in model1.parameters():
    print(parameter)
  
# for parameter in model2.parameters():
#     print(parameter)  

for parameter in model_avg.parameters():
    print(parameter)

### Test the new global model

In [ ]:

model = model_avg
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))